In [1]:
'''
预处理文件
现有文件data.jsonl。记录每一个func和idx
有文件train_sampled.txt。记录着一对func的idx和分类的类别
现在读取这两个文件,处理为train.csv。记录着一对代码和label
'''

'\n预处理文件\n现有文件data.jsonl。记录每一个func和idx\n有文件train_sampled.txt。记录着一对func的idx和分类的类别\n现在读取这两个文件,处理为train.csv。记录着一对代码和label\n'

In [1]:
idxinfo = []
with open("train_sampled.txt") as f2:
    for line in f2:
        idx1,idx2,label = line.split()
        idxinfo.append((idx1,idx2,label))
idxvalidinfo = []
with open("valid_sampled.txt") as f2:
    for line in f2:
        idx1,idx2,label = line.split()
        idxvalidinfo.append((idx1,idx2,label))
idxtestinfo = []
with open("test_sampled.txt") as f2:
    for line in f2:
        idx1,idx2,label = line.split()
        idxtestinfo.append((idx1,idx2,label))

In [2]:
'''
把data.jsonl转换为idx:func的形式
'''
import json
lst = {}
with open("data.jsonl") as f1:
        for line in f1:
            oneline = json.loads(line)
            lst[oneline["idx"]] = oneline["func"]


In [4]:
x = []
with open("test_subsTrue.jsonl") as f:
    for i in f:
        x.append(json.loads(i))
import random
y = []
for i in range(len(idxtestinfo)):
    y.append((idxtestinfo[i],x[i]))
random.shuffle(y)
idxtestinfo = []
x = []
for i in range(500):
    idxtestinfo.append(y[i][0])
    x.append(y[i][1])
with open("test_subs.jsonl","w") as f:
    for i in x:
        f.write(json.dumps(i)+"\n")

In [13]:
import pandas as pd
from tqdm import tqdm
df = pd.DataFrame(columns=["text","label"])
dict = {}
for i,v in tqdm(enumerate(idxvalidinfo)):
    onedict = {"text":None,"label":None}
    df.loc[i,"text"] = lst[v[0]] + " <SPLIT> "  + lst[v[1]] 
    df.loc[i,"label"] = int(v[2])
df.to_pickle("./valid.pkl")

4000it [00:01, 3961.47it/s]


In [5]:
import pandas as pd
from tqdm import tqdm
df = pd.DataFrame(columns=["text","label"])
dict = {}
for i,v in tqdm(enumerate(idxtestinfo)):
    onedict = {"text":None,"label":None}
    df.loc[i,"text"] = lst[v[0]] + " <SPLIT> "  + lst[v[1]] 
    df.loc[i,"label"] = int(v[2])
df.to_pickle("./test.pkl")

500it [00:00, 4707.45it/s]


In [14]:
import pandas as pd
from tqdm import tqdm
df = pd.DataFrame(columns=["text","label"])
for i,v in tqdm(enumerate(idxinfo)):
    onedict = {"text":None,"label":None}
    df.loc[i,"text"] = lst[v[0]] + " <SPLIT> "  + lst[v[1]] 
    df.loc[i,"label"] = int(v[2])
df.to_pickle("./train.pkl")

90102it [02:30, 598.25it/s] 


In [17]:
import pandas as pd
df = pd.read_pickle("train.pkl")

In [18]:
import json
adv_c =[]
with open("adv_codes.jsonl") as f:
    for line in f:
        adv_c.append(json.loads(line))

In [19]:
l = len(df)
for i in range(l,l + len(adv_c)):
    df.loc[i] = adv_c[i-l]["text"],adv_c[i-l]["label"]

In [21]:
df.to_pickle("train_adv.pkl")